In [1]:
import matplotlib.pyplot as plt
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, LabelBinarizer
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd
from random import choice

from annsa.template_sampling import *

In [2]:
import tensorflow.contrib.eager as tfe

In [3]:
tf.enable_eager_execution()

#### Import model, training function 

In [4]:
from annsa.model_classes import (dnn_model_features,
                                 DNN,
                                 save_model,
                                 train_earlystop)

## Dataset Construction

#### Load dataset

In [5]:
background_dataset = pd.read_csv('../../source-interdiction/training_testing_data/background_template_dataset.csv')
source_dataset = pd.read_csv('../../source-interdiction/training_testing_data/shielded_templates_200kev_dataset.csv')

#### Dataset details

In [6]:
print('sourcedist: ' + str(sorted(set(source_dataset['sourcedist']))))
print('sourceheight: ' + str(sorted(set(source_dataset['sourceheight']))))
print('alum shieldingdensity: ' + str(sorted(set(source_dataset[source_dataset['shielding']=='alum']['shieldingdensity']))))
print('iron shieldingdensity: ' + str(sorted(set(source_dataset[source_dataset['shielding']=='iron']['shieldingdensity']))))
print('lead shieldingdensity: ' + str(sorted(set(source_dataset[source_dataset['shielding']=='lead']['shieldingdensity']))))
print('fwhm: ' + str(sorted(set(source_dataset['fwhm']))))

sourcedist: [50.0, 112.5, 175.0, 237.5, 300.0]
sourceheight: [50.0, 75.0, 100.0, 125.0, 150.0]
alum shieldingdensity: [1.82, 4.18, 7.49, 13.16]
iron shieldingdensity: [1.53, 3.5, 6.28, 11.02]
lead shieldingdensity: [0.22, 0.51, 0.92, 1.61]
fwhm: [6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0]


#### Customize dataset

In [7]:
source_dataset = source_dataset[source_dataset['fwhm']==7.5]
source_dataset = source_dataset[source_dataset['sourcedist']==175.0]
source_dataset = source_dataset[source_dataset['sourceheight']==100.0]

# remove 80% shielding
source_dataset = source_dataset[source_dataset['shieldingdensity']!=13.16]
source_dataset = source_dataset[source_dataset['shieldingdensity']!=11.02]
source_dataset = source_dataset[source_dataset['shieldingdensity']!=1.61]

# remove 60% shielding
source_dataset = source_dataset[source_dataset['shieldingdensity']!=7.49]
source_dataset = source_dataset[source_dataset['shieldingdensity']!=6.28]
source_dataset = source_dataset[source_dataset['shieldingdensity']!=0.92]

# remove 40% shielding
source_dataset = source_dataset[source_dataset['shieldingdensity']!=4.18]
source_dataset = source_dataset[source_dataset['shieldingdensity']!=3.5]
source_dataset = source_dataset[source_dataset['shieldingdensity']!=0.51]

# remove 20% shielding
source_dataset = source_dataset[source_dataset['shieldingdensity']!=1.82]
source_dataset = source_dataset[source_dataset['shieldingdensity']!=1.53]
source_dataset = source_dataset[source_dataset['shieldingdensity']!=0.22]

#### Remove empty spectra

In [8]:
zero_count_indicies = np.argwhere(np.sum(source_dataset.values[:,6:],axis=1) == 0).flatten()

print('indicies dropped: ' +str(zero_count_indicies))

source_dataset.drop(source_dataset.index[zero_count_indicies], inplace=True)

indicies dropped: []


#### Add empty spectra for background 

In [9]:
blank_spectra = []
for fwhm in set(source_dataset['fwhm']):
    num_examples = source_dataset[(source_dataset['fwhm']==fwhm) &
                                  (source_dataset['isotope']==source_dataset['isotope'].iloc()[0])].shape[0]
    for k in range(num_examples):
        blank_spectra_tmp = [0]*1200
        blank_spectra_tmp[5] = fwhm
        blank_spectra_tmp[0] = 'background'
        blank_spectra_tmp[3] = 'background'
        blank_spectra.append(blank_spectra_tmp)

source_dataset = source_dataset.append(pd.DataFrame(blank_spectra,
                                                    columns=source_dataset.columns))

#### Create dataset from spectra

In [10]:
spectra_dataset = source_dataset.values[:,5:].astype('float64')
all_keys = source_dataset['isotope'].values

## Define Training Parameters

#### Define online data augmentation

In [11]:
def integration_time():
    return np.random.uniform(60,600)

def background_cps():
    return np.random.poisson(200)

def signal_to_background():
    return np.random.uniform(0.5,2)

def calibration():
    return [np.random.uniform(0,10),
            np.random.uniform(2500/3000,3500/3000),
            0]

online_data_augmentation = online_data_augmentation_vanilla(background_dataset,
                                background_cps,
                                integration_time,
                                signal_to_background,
                                calibration,)

## Load testing dataset 

In [12]:
testing_dataset = np.load('../dataset_generation/testing_dataset_200keV_1000.npy')

In [13]:
testing_spectra = np.random.poisson(np.add(testing_dataset.item()['sources'], testing_dataset.item()['backgrounds']))
testing_keys = testing_dataset.item()['keys']

mlb=LabelBinarizer()

all_keys_binarized = mlb.fit_transform(all_keys.reshape([all_keys.shape[0],1]))
testing_keys_binarized = mlb.transform(testing_keys)
training_keys_binarized = mlb.transform(all_keys)

## Train network

### Define hyperparameters

In [14]:
number_hyperparameters_to_search = 256
earlystop_errors_test = []
model_id='DNN-onlinedataaugeasy'

In [15]:
def make_model():
    '''
    Makes a random model given some parameters.

    '''
    number_layers = choice([1, 2, 3])
    dense_nodes = 2**np.random.randint(5, 10, number_layers)
    dense_nodes = np.sort(dense_nodes)
    dense_nodes = np.flipud(dense_nodes)
    model_features = dnn_model_features(
        learining_rate=10**np.random.uniform(-4,-1),
        l2_regularization_scale=10**np.random.uniform(-2,0),
        dropout_probability=np.random.uniform(0,1),
        batch_size=2**np.random.randint(4,10),
        output_size=training_keys_binarized.shape[1],
        dense_nodes=dense_nodes,
        activation_function=choice([tf.nn.tanh,tf.nn.relu,tf.nn.sigmoid]),
        scaler=choice([make_pipeline(FunctionTransformer(np.log1p, validate=True)),
                       make_pipeline(FunctionTransformer(np.sqrt, validate=True))]))

    model = DNN(model_features)

    return model, model_features 

### Search hyperparameters

In [17]:
testing_errors = []

for network_id in range(number_hyperparameters_to_search):
    # reset model on each iteration
    model, model_features = make_model()
    optimizer = tf.train.AdamOptimizer(model_features.learining_rate)
    
    costfunction_errors_tmp, earlystop_errors_tmp = train_earlystop(
            training_data=spectra_dataset,
            training_keys=training_keys_binarized,
            testing_data=testing_spectra,
            testing_keys=testing_keys_binarized,
            model=model,
            optimizer=optimizer,
            num_epochs=500,
            obj_cost=model.cross_entropy,
            earlystop_cost_fn=model.f1_error,
            earlystop_patience=10,
            verbose=True,
            fit_batch_verbose=1,
            data_augmentation=online_data_augmentation)
    testing_errors.append(earlystop_errors_tmp)

    # np.save('./final-models/final_test_errors_'+model_id, training_errors)
    # model.save_weights('./final-models/'+model_id+'_checkpoint_'+str(network_id))
    network_id += 1 

Epoch 1: CostFunc loss: 0.00 7.27, EarlyStop loss: 0.00 0.94
Epoch 2: CostFunc loss: 0.00 7.23, EarlyStop loss: 0.00 0.94
Epoch 3: CostFunc loss: 0.00 7.18, EarlyStop loss: 0.00 0.94
Epoch 4: CostFunc loss: 0.00 7.14, EarlyStop loss: 0.00 0.93
Epoch 5: CostFunc loss: 0.00 7.10, EarlyStop loss: 0.00 0.94
Epoch 6: CostFunc loss: 0.00 7.06, EarlyStop loss: 0.00 0.94
Epoch 7: CostFunc loss: 0.00 7.01, EarlyStop loss: 0.00 0.94
Epoch 8: CostFunc loss: 0.00 6.98, EarlyStop loss: 0.00 0.94
Epoch 9: CostFunc loss: 0.00 6.94, EarlyStop loss: 0.00 0.94
Epoch 10: CostFunc loss: 0.00 6.90, EarlyStop loss: 0.00 0.94
Epoch 11: CostFunc loss: 0.00 6.86, EarlyStop loss: 0.00 0.94
Epoch 12: CostFunc loss: 0.00 6.83, EarlyStop loss: 0.00 0.94
Epoch 13: CostFunc loss: 0.00 6.79, EarlyStop loss: 0.00 0.94
Test error at early stop: Objectives fctn: 7.14 Early stopfctn: 7.14
Epoch 1: CostFunc loss: 0.00 30.21, EarlyStop loss: 0.00 0.97
Epoch 2: CostFunc loss: 0.00 28.47, EarlyStop loss: 0.00 0.97
Epoch 3: C

Epoch 1: CostFunc loss: 0.00 142.65, EarlyStop loss: 0.00 0.97
Epoch 2: CostFunc loss: 0.00 135.90, EarlyStop loss: 0.00 0.97
Epoch 3: CostFunc loss: 0.00 138.68, EarlyStop loss: 0.00 0.97
Epoch 4: CostFunc loss: 0.00 153.42, EarlyStop loss: 0.00 0.97
Epoch 5: CostFunc loss: 0.00 141.32, EarlyStop loss: 0.00 0.97
Epoch 6: CostFunc loss: 0.00 126.17, EarlyStop loss: 0.00 0.95
Epoch 7: CostFunc loss: 0.00 115.00, EarlyStop loss: 0.00 0.99
Epoch 8: CostFunc loss: 0.00 118.76, EarlyStop loss: 0.00 0.99
Epoch 9: CostFunc loss: 0.00 124.90, EarlyStop loss: 0.00 0.95
Epoch 10: CostFunc loss: 0.00 121.54, EarlyStop loss: 0.00 0.98
Epoch 11: CostFunc loss: 0.00 116.67, EarlyStop loss: 0.00 1.00
Epoch 12: CostFunc loss: 0.00 124.58, EarlyStop loss: 0.00 0.94
Epoch 13: CostFunc loss: 0.00 124.32, EarlyStop loss: 0.00 0.97
Epoch 14: CostFunc loss: 0.00 123.93, EarlyStop loss: 0.00 0.94
Epoch 15: CostFunc loss: 0.00 120.98, EarlyStop loss: 0.00 0.97
Epoch 16: CostFunc loss: 0.00 117.15, EarlyStop l

Epoch 15: CostFunc loss: 0.00 9.13, EarlyStop loss: 0.00 0.91
Epoch 16: CostFunc loss: 0.00 8.48, EarlyStop loss: 0.00 0.88
Epoch 17: CostFunc loss: 0.00 7.91, EarlyStop loss: 0.00 0.92
Epoch 18: CostFunc loss: 0.00 7.38, EarlyStop loss: 0.00 0.91
Epoch 19: CostFunc loss: 0.00 6.92, EarlyStop loss: 0.00 0.88
Epoch 20: CostFunc loss: 0.00 6.55, EarlyStop loss: 0.00 0.83
Epoch 21: CostFunc loss: 0.00 6.22, EarlyStop loss: 0.00 0.88
Epoch 22: CostFunc loss: 0.00 5.91, EarlyStop loss: 0.00 0.88
Epoch 23: CostFunc loss: 0.00 5.59, EarlyStop loss: 0.00 0.88
Epoch 24: CostFunc loss: 0.00 5.33, EarlyStop loss: 0.00 0.78
Epoch 25: CostFunc loss: 0.00 5.15, EarlyStop loss: 0.00 0.86
Epoch 26: CostFunc loss: 0.00 4.95, EarlyStop loss: 0.00 0.83
Epoch 27: CostFunc loss: 0.00 4.80, EarlyStop loss: 0.00 0.84
Epoch 28: CostFunc loss: 0.00 4.64, EarlyStop loss: 0.00 0.87
Epoch 29: CostFunc loss: 0.00 4.52, EarlyStop loss: 0.00 0.83
Epoch 30: CostFunc loss: 0.00 4.39, EarlyStop loss: 0.00 0.83
Epoch 31

Epoch 1: CostFunc loss: 0.00 98.18, EarlyStop loss: 0.00 0.97
Epoch 2: CostFunc loss: 0.00 96.83, EarlyStop loss: 0.00 0.97
Epoch 3: CostFunc loss: 0.00 95.50, EarlyStop loss: 0.00 0.97
Epoch 4: CostFunc loss: 0.00 94.17, EarlyStop loss: 0.00 0.97
Epoch 5: CostFunc loss: 0.00 92.87, EarlyStop loss: 0.00 0.97
Epoch 6: CostFunc loss: 0.00 91.58, EarlyStop loss: 0.00 0.97
Epoch 7: CostFunc loss: 0.00 90.30, EarlyStop loss: 0.00 0.96
Epoch 8: CostFunc loss: 0.00 89.04, EarlyStop loss: 0.00 0.95
Epoch 9: CostFunc loss: 0.00 87.80, EarlyStop loss: 0.00 0.95
Epoch 10: CostFunc loss: 0.00 86.57, EarlyStop loss: 0.00 0.96
Epoch 11: CostFunc loss: 0.00 85.36, EarlyStop loss: 0.00 0.96
Epoch 12: CostFunc loss: 0.00 84.16, EarlyStop loss: 0.00 0.97
Epoch 13: CostFunc loss: 0.00 82.98, EarlyStop loss: 0.00 0.97
Epoch 14: CostFunc loss: 0.00 81.81, EarlyStop loss: 0.00 0.97
Epoch 15: CostFunc loss: 0.00 80.66, EarlyStop loss: 0.00 0.97
Epoch 16: CostFunc loss: 0.00 79.52, EarlyStop loss: 0.00 0.97
E

Epoch 26: CostFunc loss: 0.00 7.66, EarlyStop loss: 0.00 0.97
Epoch 27: CostFunc loss: 0.00 7.48, EarlyStop loss: 0.00 0.97
Epoch 28: CostFunc loss: 0.00 7.31, EarlyStop loss: 0.00 0.97
Epoch 29: CostFunc loss: 0.00 7.15, EarlyStop loss: 0.00 0.97
Epoch 30: CostFunc loss: 0.00 6.98, EarlyStop loss: 0.00 0.97
Epoch 31: CostFunc loss: 0.00 6.79, EarlyStop loss: 0.00 0.97
Epoch 32: CostFunc loss: 0.00 6.62, EarlyStop loss: 0.00 0.96
Epoch 33: CostFunc loss: 0.00 6.47, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 8.03 Early stopfctn: 8.03
Epoch 1: CostFunc loss: 0.00 4.08, EarlyStop loss: 0.00 0.97
Epoch 2: CostFunc loss: 0.00 3.96, EarlyStop loss: 0.00 0.95
Epoch 3: CostFunc loss: 0.00 3.91, EarlyStop loss: 0.00 0.97
Epoch 4: CostFunc loss: 0.00 3.89, EarlyStop loss: 0.00 0.97
Epoch 5: CostFunc loss: 0.00 3.81, EarlyStop loss: 0.00 0.97
Epoch 6: CostFunc loss: 0.00 3.80, EarlyStop loss: 0.00 0.97
Epoch 7: CostFunc loss: 0.00 3.82, EarlyStop loss: 0.00 0.97
Epoch 8:

KeyboardInterrupt: 